In [ ]:
#download das bases - arquivo completo
!wget -p https://www.anatel.gov.br/dadosabertos/paineis_de_dados/acessos/acessos_telefonia_movel.zip

--2021-07-17 19:25:19--  https://www.anatel.gov.br/dadosabertos/paineis_de_dados/acessos/acessos_telefonia_movel.zip
Resolving www.anatel.gov.br (www.anatel.gov.br)... 200.0.81.81, 2801:80:c90:c1da:da0:2312:2000:1331
Connecting to www.anatel.gov.br (www.anatel.gov.br)|200.0.81.81|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 687710666 (656M) [application/x-zip-compressed]
Saving to: ‘www.anatel.gov.br/dadosabertos/paineis_de_dados/acessos/acessos_telefonia_movel.zip’

sabertos/paineis_de  39%[======>             ] 260.29M   914KB/s    eta 7m 24s ^C


In [ ]:
import pandas as pd
import numpy as np
import os
from zipfile import ZipFile

In [ ]:
pasta = '/content/www.anatel.gov.br/dadosabertos/paineis_de_dados/acessos'
mobile = os.path.join(pasta, 'acessos_telefonia_movel.zip')

In [ ]:
#decodificando os arquivos (2005-2018)
  #Anatel separa em tecnologia(tipo de conexão) e modalidade (tipo de planos: pré e pós pago)
with ZipFile(mobile) as z:
  with z.open(f'Acessos_Telefonia_Movel_2005-2018_Tecnologia.csv') as f:
   tm_tec = pd.read_csv(f,sep=';', encoding='utf-8')

In [ ]:
#renomeando/padronizando variaveis ref. tm_tec (telefonia móvel - tecnologia)
tm_tec.rename(columns={'Ano': 'ano','Mês':'mes', 'Grupo Econômico':'grupo_economico', 'Empresa':'empresa',
                 'CNPJ':'cnpj', 'Porte da Prestadora':'porte_empresa', 'UF':'sigla_uf', 'Região PGO':'outorga', 
                 'Código Nacional':'ddd', 'Tecnologia':'tecnologia', 'Tecnologia Geração':'sinal', 'Acessos':'acessos'}, inplace=True) 

In [ ]:
#exclusão de dados NaN (ddd & sigla_uf), início das observações completas em 2009
tm_tec.dropna(inplace=True)

In [ ]:
#exclusão de variáveis duplicadas
tm_tec['acessos_total'] = tm_tec.groupby(['ano', 'mes', 'ddd', 'cnpj', 'tecnologia', 'sinal'])['acessos'].transform(np.sum)
tm_tec.sort_values(['ano', 'mes', 'ddd', 'cnpj', 'tecnologia', 'sinal'], inplace=True)
tm_tec.drop_duplicates(subset=['ano', 'mes', 'ddd', 'cnpj', 'tecnologia', 'sinal'], keep='first', inplace=True)
tm_tec.drop('acessos', axis=1, inplace=True)
tm_tec.rename(columns={'acessos_total':'acessos'}, inplace=True)

In [ ]:
with ZipFile(mobile) as z:
  with z.open(f'Acessos_Telefonia_Movel_2005-2018_Modalidade.csv') as f:
   tm_mod = pd.read_csv(f,sep=';', encoding='utf-8')

In [ ]:
#renomeando/padronizando variaveis ref. tm_mod (telefonia_movel - modalidade)
tm_mod.rename(columns={'Ano': 'ano','Mês':'mes', 'Grupo Econômico':'grupo_economico', 'Empresa':'empresa',
                 'CNPJ':'cnpj', 'Porte da Prestadora':'porte_empresa', 'UF':'sigla_uf', 'Região PGO':'outorga', 
                 'Código Nacional':'ddd', 'Modalidade de Cobrança':'modalidade', 'Acessos':'acessos'}, inplace=True) 

In [ ]:
#exclusão de variáveis duplicadas
tm_mod['acessos_total'] = tm_mod.groupby(['ano', 'mes', 'ddd', 'cnpj', 'modalidade'])['acessos'].transform(np.sum)
tm_mod.sort_values(['ano', 'mes', 'ddd', 'cnpj', 'modalidade'], inplace=True)
tm_mod.drop_duplicates(subset=['ano', 'mes', 'ddd', 'cnpj', 'modalidade'], keep='first', inplace=True)
tm_mod.drop('acessos', axis=1, inplace=True)
tm_mod.rename(columns={'acessos_total':'acessos'}, inplace=True)

In [ ]:
#drop de colunas não necessárias
tm_tec.drop(['grupo_economico', 'outorga'], axis=1, inplace=True)
tm_mod.drop(['grupo_economico', 'outorga'], axis=1, inplace=True)

In [ ]:
#ordenação das colunas
tm_tec = tm_tec[['ano', 'mes', 'sigla_uf', 'ddd', 'cnpj', 'empresa',  'porte_empresa',  'tecnologia', 'sinal', 'acessos']]
tm_mod = tm_mod[['ano', 'mes', 'sigla_uf', 'ddd', 'cnpj', 'empresa',  'porte_empresa',  'modalidade', 'acessos']]

In [ ]:
#transformação da variável de int64 pra string
tm_tec['ddd'] = pd.to_numeric(tm_tec['ddd'], downcast='integer').astype(str)
tm_tec['cnpj'] = tm_tec['cnpj'].astype(str)

In [ ]:
#transformação da variável de int64 pra string
tm_mod['ddd'] = pd.to_numeric(tm_mod['ddd'], downcast='integer').astype(str)
tm_mod['cnpj'] = tm_mod['cnpj'].astype(str)

In [ ]:
#salvar arquivos no Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#exportação arquivo csv na memória do Colab
#tm_tec.to_csv('/content/telefonia_movel_microdados_2009-2018_tecnologia.csv', index=False)
#tm_mod.to_csv('/content/telefonia_movel_microdados_2005-2018_modalidade.csv', index=False)
#ou salvar arquivos direto no Drive
tm_tec.to_csv('/content/drive/MyDrive/br_anatel/telefonia_movel/output/telefonia_movel_microdados_tecnologia_2009-2018.csv', index=False)
tm_mod.to_csv('/content/drive/MyDrive/br_anatel/telefonia_movel/output/telefonia_movel_microdados_modalidade_2005-2018.csv', index=False)

In [ ]:
#decodificando os arquivos (2019-2021)
  #Anatel separa por ano/semestre
with ZipFile(mobile) as z:
  with z.open(f'Acessos_Telefonia_Movel_201901-201906.csv') as f:
    tm_1 = pd.read_csv(f,sep=';', encoding='utf-8')

In [ ]:
with ZipFile(mobile) as z:
  with z.open(f'Acessos_Telefonia_Movel_201907-201912.csv') as f:
    tm_2 = pd.read_csv(f,sep=';', encoding='utf-8')

In [ ]:
with ZipFile(mobile) as z:
  with z.open(f'Acessos_Telefonia_Movel_202001-202006.csv') as f:
    tm_3 = pd.read_csv(f,sep=';', encoding='utf-8')

In [ ]:
with ZipFile(mobile) as z:
  with z.open(f'Acessos_Telefonia_Movel_202007-202012.csv') as f:
    tm_4 = pd.read_csv(f,sep=';', encoding='utf-8')

In [ ]:
with ZipFile(mobile) as z:
  with z.open(f'Acessos_Telefonia_Movel_2021.csv') as f:
    tm_5 = pd.read_csv(f,sep=';', encoding='utf-8')

In [ ]:
#renomeando as variáveis dos diferentes dataframes e excluindo variáveis desnecessárias
df_list = [tm_1, tm_2, tm_3, tm_4, tm_5]

for i in df_list:
    i.rename(columns={'Ano': 'ano','Mês':'mes', 'Grupo Econômico':'grupo_economico', 'Empresa':'empresa',
                       'CNPJ':'cnpj', 'Porte da Prestadora':'porte_empresa', 'UF':'sigla_uf', 'Município':'municipio',
                       'Código IBGE Município':'id_municipio', 'Código Nacional':'ddd', 'Código Nacional (Chip)':'ddd_chip',
                       'Modalidade de Cobrança':'modalidade', 'Tecnologia':'tecnologia', 'Tecnologia Geração':'sinal', 
                       'Tipo de Pessoa':'pessoa', 'Tipo de Produto':'produto', 'Acessos':'acessos'}, inplace=True)
    i.drop(['grupo_economico', 'municipio', 'ddd_chip'], axis=1, inplace=True)

NameError: ignored

In [ ]:
#junção das bases
tm_mun = tm_1.append([tm_2, tm_3, tm_4, tm_5], ignore_index=True)

In [ ]:
#tm_mun.to_csv('/content/telefonia_movel_microdados_2019-2021.csv', index=False) #salvar na memória do Colab
tm_mun.to_csv('/content/drive/MyDrive/br_anatel/telefonia_movel/output/telefonia_movel_microdados_2019-2021.csv', index=False)

In [ ]:
#os dados possuem duplicações mostrando diferentes resultados para acesso
#o agrupamento é feito pelas variáveis de identificação
tm_mun['acessos_total'] = tm_mun.groupby(['ano', 'mes', 'sigla_uf', 'id_municipio', 'ddd', 'cnpj', 'empresa', 'porte_empresa', 'tecnologia', 'sinal', 'modalidade', 'pessoa', 'produto'])['acessos'].transform(np.sum)

In [ ]:
#após ordenamento das observações, se mantém somente 1 linha que identifique as observações
tm_mun.sort_values(['ano', 'mes', 'sigla_uf', 'id_municipio', 'ddd', 'cnpj', 'empresa', 'porte_empresa', 'tecnologia', 'sinal', 'modalidade', 'pessoa', 'produto'], inplace=True)
tm_mun.drop_duplicates(subset=['ano', 'mes', 'sigla_uf', 'id_municipio', 'ddd', 'cnpj', 'empresa', 'porte_empresa', 'tecnologia', 'sinal', 'modalidade', 'pessoa', 'produto'], keep='first', inplace=True)

In [ ]:
#exclui-se a coluna de acessos e renomeia
tm_mun.drop('acessos', axis=1, inplace=True)
tm_mun.rename(columns={'acessos_total':'acessos'}, inplace=True)

In [ ]:
#ordenação das variáveis
tm_mun = tm_mun[['ano', 'mes', 'sigla_uf', 'id_municipio', 'ddd', 'cnpj', 'empresa', 'porte_empresa', 'tecnologia', 'sinal', 'modalidade', 'pessoa', 'produto', 'acessos']]

In [ ]:
#tipifcação da base
tm_mun['id_municipio'] = tm_mun['id_municipio'].astype(str)
tm_mun['ddd'] = pd.to_numeric(tm_mun['ddd'], downcast='integer').astype(str)
tm_mun['cnpj'] = tm_mun['cnpj'].astype(str)

In [ ]:
#tm_mun.to_csv('/content/telefonia_movel_municipio_2019-2021.csv', index=False) #salvar na memória do Colab
tm_mun.to_csv('/content/drive/MyDrive/br_anatel/telefonia_movel/output/telefonia_movel_municipio_2019-2021.csv', index=False)